In [36]:
#!/usr/bin/python

import uproot
from icecream import ic
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import subprocess
import os
import time
import shutil
from shutil import copyfile


In [ ]:
M = 0.938272081
me = 0.5109989461 * 0.001
ebeam = 10.604
pbeam = np.sqrt(ebeam * ebeam - me * me)
beam = [0, 0, pbeam]
target = [0, 0, 0]

In [35]:
data_file = "skim8_005032_filtered.root"
file = uproot.open(data_file)
        
tree = file["T"]   
print(tree.keys())

['nmb', 'Pp', 'Ppx', 'Ppy', 'Ppz', 'Ptheta', 'Pphi', 'Pvx', 'Pvy', 'Pvz', 'Pvt', 'Pbeta', 'Pstat', 'PorigIndx', 'nml', 'Ep', 'Epx', 'Epy', 'Epz', 'Etheta', 'Ephi', 'Evx', 'Evy', 'Evz', 'Evt', 'Ebeta', 'Estat', 'EorigIndx', 'nmg', 'Gp', 'Gpx', 'Gpy', 'Gpz', 'Gtheta', 'Gphi', 'Gvx', 'Gvy', 'Gvz', 'Gvt', 'Gbeta', 'Gstat', 'GorigIndx', 'beamQ', 'liveTime', 'startTime', 'RFTime', 'helicity', 'helicityRaw', 'EventNum', 'RunNum', 'Q2', 'Nu', 'q', 'qx', 'qy', 'qz', 'W2', 'xB', 't']


In [37]:
# initialize df
df_electron = pd.DataFrame()
df_proton = pd.DataFrame()
df_gamma = pd.DataFrame()

In [49]:
for key in ["RunNum","EventNum",'helicity','helicityRaw',"Epx", "Epy", "Epz", "Evz", "Etheta","Ephi"]:
    df_electron[key] = tree[key].array(library="pd")

for key in ["Ppx", "Ppy", "Ppz", "Pvz", "Pstat", "PorigIndx", "Ptheta","Pphi"]:
    df_proton[key] = tree[key].array(library="pd")
    

for key in ["Gpx", "Gpy", "Gpz", "Gstat", "GorigIndx", "Gtheta","Gphi"]:
    df_gamma[key] = tree[key].array(library="pd")
    
print(df_electron)
print(df_proton)
print(df_gamma)


                     Epx       Epy       Epz       Evz    Etheta      Ephi  \
entry subentry                                                               
0     0         0.128060  0.794708  2.058435  0.020737  0.372771  1.411029   
1     0         0.609048  0.953577  5.935979 -3.554013  0.188354  1.002407   
2     0        -0.969444 -0.070090  6.233665 -1.064051  0.154678 -3.069418   
3     0         0.272029  0.575523  1.988892 -4.445016  0.309762  1.129256   
4     0         0.513050 -0.645339  4.399658 -3.606798  0.185237 -0.899106   
...                  ...       ...       ...       ...       ...       ...   
57155 0         0.591130  1.022972  7.913909 -4.363057  0.148198  1.046818   
57156 0         0.615304 -1.258897  8.077948 -0.946698  0.171753 -1.116177   
57157 0         0.548618 -0.780266  6.480116 -0.783584  0.146144 -0.957982   
57158 0        -0.307810 -0.686286  4.631073  0.396530  0.161009 -1.992416   
57159 0        -0.153188 -0.607261  2.778714 -0.928987  0.221682

In [45]:
# change the data type to double, Sangbaek knows why
df_electron = df_electron.astype({"Epx": float, "Epy": float, "Epz": float})
df_proton = df_proton.astype({"Ppx": float, "Ppy": float, "Ppz": float})
df_gamma = df_gamma.astype({"Gpx": float, "Gpy": float, "Gpz": float})

In [46]:
#Get proper indexing before merging
df_electron['event'] = df_electron.index.get_level_values('entry')
df_proton['event'] = df_proton.index.get_level_values('entry')
df_gamma['event'] = df_gamma.index.get_level_values('entry')

In [48]:
print(df_electron)

                     Epx       Epy       Epz       Evz    Etheta      Ephi  \
entry subentry                                                               
0     0         0.128060  0.794708  2.058435  0.020737  0.372771  1.411029   
1     0         0.609048  0.953577  5.935979 -3.554013  0.188354  1.002407   
2     0        -0.969444 -0.070090  6.233665 -1.064051  0.154678 -3.069418   
3     0         0.272029  0.575523  1.988892 -4.445016  0.309762  1.129256   
4     0         0.513050 -0.645339  4.399658 -3.606798  0.185237 -0.899106   
...                  ...       ...       ...       ...       ...       ...   
57155 0         0.591130  1.022972  7.913909 -4.363057  0.148198  1.046818   
57156 0         0.615304 -1.258897  8.077948 -0.946698  0.171753 -1.116177   
57157 0         0.548618 -0.780266  6.480116 -0.783584  0.146144 -0.957982   
57158 0        -0.307810 -0.686286  4.631073  0.396530  0.161009 -1.992416   
57159 0        -0.153188 -0.607261  2.778714 -0.928987  0.221682